In [91]:
import pyro
import pyro.optim as optim
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from torch.distributions import constraints
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing

In [92]:
pyro.set_rng_seed(1)
pyro.enable_validation(True)
pyro.clear_param_store()

In [93]:
# loading datasets
df = pd.read_csv('wine.data', header=None)
X = df.iloc[:,1:]
y = df.iloc[:,0].astype(int)
y = y - 1

In [94]:
def model(X):
    # rejestrujemy zmienną do przestrzeni optymalizacji (store pyro)
    mu_param = pyro.param("mu", torch.zeros_like(naive_bayes.current_class_probs))
    # ograniczamy wartości do nieujemnych
    sigma_param = pyro.param("sigma", torch.ones_like(naive_bayes.current_class_probs), constraint=constraints.positive)
    params = pyro.distributions.Normal(loc=mu_param, scale=sigma_param).to_event(1)
    with pyro.plate("map", len(X)):
        pyro.sample("probs", params, obs=X)

def guide(X):
    # rejestrujemy zmienną do przestrzeni optymalizacji (store pyro)
    mu_param = pyro.param("mu", torch.zeros_like(naive_bayes.current_class_probs))
    # ograniczamy wartości do nieujemnych
    sigma_param = pyro.param("sigma", torch.ones_like(naive_bayes.current_class_probs), constraint=constraints.positive)
    probs_prior = pyro.distributions.Normal(loc=mu_param, scale=sigma_param).to_event(1)
    return pyro.sample("probs", probs_prior, infer={'is_auxiliary': True})

def train(X):
    pyro.clear_param_store()
    num_iterations=5000
    optim = pyro.optim.Adam({"lr": 0.03})
    svi = pyro.infer.SVI(model, guide, optim, loss=pyro.infer.Trace_ELBO(), num_samples=len(X))
    losses = list()
    t=tqdm(range(num_iterations))
    for j in t:
        loss = svi.step(X)
        losses.append(loss)
        t.set_postfix(loss=loss)
    return (svi, losses)

In [95]:
class NaiveBayesClassifier:
    def __init__(self, threshold=0.8):
        self.threshold = threshold
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        
        self.available_y = np.unique(y)
        self.num_features = X.shape[1]
        self._count_y_prob()
        self.params_for_probs = list()
        
        for target in self.available_y:
            self.X_current_class = torch.from_numpy(X[y==target])
            self.current_class_probs = torch.from_numpy(np.random.randn(self.num_features))
            train(self.X_current_class)
            mu = pyro.param("mu")
            sigma = pyro.param("sigma")
            sigma2 = sigma*sigma # pyro uses std instead of var
            self.params_for_probs.append(torch.stack([mu, sigma2], dim=0))
            
        for i in range(len(self.params_for_probs)):
            self.params_for_probs[i] = self.params_for_probs[i].detach().numpy()
            
    def _count_y_prob(self):
        total_quantity = len(self.y)
        self.p_y = [np.count_nonzero(self.y == i) / total_quantity for i in self.available_y]
    
    def predict(self, X):
        """Returns -1 if classifier is not sure of prediciton."""
        predicted = list()
        for i in range(len(X)):
            predicted.append(self._predict_one_example(i, X[i, :]))
        return np.asarray(predicted)

    def _predict_one_example(self, i: int, x: np.ndarray):
        certainity_for_ys = list()
        for y in self.available_y:  # for every class
            certainity_for_ys.append(self.p_y[y])
            for i in range(len(x)):  # for every feature
                certainity_for_ys[-1] *= self._p_xi_on_condition_y(i, y, x[i])
        certainity_for_ys = self._normalize_values(certainity_for_ys)
        if max(certainity_for_ys) > self.threshold:
            return self.available_y[certainity_for_ys.index(max(certainity_for_ys))]
        else:
            print("I'm not sure.")
            return -1
    
    def _p_xi_on_condition_y(self, feature_index, y_index, x_i):
        multiplier = 1 / np.sqrt(2 * np.pi * self.params_for_probs[y_index][1, feature_index])
        exp = - (x_i - self.params_for_probs[y_index][0, feature_index]) ** 2 / (2 * self.params_for_probs[y_index][1, feature_index])
        return multiplier * np.power(np.e, exp)
    
    def _normalize_values(self, v):
        sum_of_values = sum(v)
        out_vals = v / sum_of_values
        return list(out_vals)

In [96]:
naive_bayes = NaiveBayesClassifier()
# naive_bayes.fit(X, y)

In [97]:
def crossval_research(data, target):
    splitter = StratifiedKFold(n_splits=7, shuffle=True)
    split_set_generator = splitter.split(data, target)

    # trainning and testing
    y_pred = list()
    y_true = list()

    #train_indices, test_indices = next(split_set_generator)
    for train_indices, test_indices in split_set_generator:
        X_train = data[train_indices]
        Y_train = target[train_indices]
        naive_bayes.fit(X_train, Y_train)
        y_pred.extend(naive_bayes.predict(data[test_indices]))
        y_true.extend(target[test_indices])

    whole_ds_len = len(y_pred)
    # removing examples that bayes is uncertain of
    for i in range(len(y_pred)-1,0,-1):
        if y_pred[i] == -1:
            del y_pred[i]
            del y_true[i]
    removed_uncertain_len = len(y_pred)
            
    confusion = metrics.confusion_matrix(y_true, y_pred)
    accuracy = metrics.accuracy_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred, average=None)
    recall = metrics.recall_score(y_true, y_pred, average=None)
    f1_score = metrics.f1_score(y_true, y_pred, average=None)
    
    print("Number of examples classified: {} of {}".format(removed_uncertain_len, whole_ds_len))
    print("It is {}%".format(float(removed_uncertain_len)/whole_ds_len*100.0))

    return {"confusion": confusion, "accuracy": accuracy, "precision": precision,
            "recall": recall, "f1_score": f1_score}

In [ ]:
class_metrics = crossval_research(X.values, y.values)

I'm not sure.
I'm not sure.


I'm not sure.


I'm not sure.
I'm not sure.
I'm not sure.


I'm not sure.


I'm not sure.


In [ ]:
print(class_metrics)

### Reasearch of model and guide

In [4]:
X_class_0 = X[y==0].values
print(X_class_0.mean(0))
print(X_class_0.std(0))

means = torch.zeros_like(torch.from_numpy(X_class_0[0,:]))
stds = torch.ones_like(torch.from_numpy(X_class_0[0,:]))

X_class_0 = torch.from_numpy(X_class_0)

[1.37447458e+01 2.01067797e+00 2.45559322e+00 1.70372881e+01
 1.06338983e+02 2.84016949e+00 2.98237288e+00 2.90000000e-01
 1.89932203e+00 5.52830508e+00 1.06203390e+00 3.15779661e+00
 1.11571186e+03]
[4.58192306e-01 6.82688763e-01 2.25232619e-01 2.52465123e+00
 1.04095949e+01 3.36076522e-01 3.94110623e-01 6.94530691e-02
 4.08601851e-01 1.22803157e+00 1.15491282e-01 3.54037572e-01
 2.19635449e+02]


In [23]:
def model(X):
    # rejestrujemy zmienną do przestrzeni optymalizacji (store pyro)
    mu_param = pyro.param("mu", torch.zeros_like(means))
    # ograniczamy wartości do nieujemnych
    sigma_param = pyro.param("sigma", torch.ones_like(stds), constraint=constraints.positive)
    params = pyro.distributions.Normal(loc=mu_param, scale=sigma_param)
    #with pyro.plate("map", len(X)):
    pyro.sample("probs", params, obs=X)

def guide(X):
    # rejestrujemy zmienną do przestrzeni optymalizacji (store pyro)
    mu_param = pyro.param("mu", torch.zeros_like(means))
    # ograniczamy wartości do nieujemnych
    sigma_param = pyro.param("sigma", torch.ones_like(stds), constraint=constraints.positive)
    probs_prior = pyro.distributions.Normal(loc=mu_param, scale=sigma_param)
    return pyro.sample("probs", probs_prior, infer={'is_auxiliary': True})

def train(X):
    pyro.clear_param_store()
    num_iterations=1000
    optim = pyro.optim.Adam({"lr": 0.05})
    svi = pyro.infer.SVI(model, guide, optim, loss=pyro.infer.Trace_ELBO(), num_samples=len(X))
    losses = list()
    t=tqdm(range(num_iterations))
    for j in t:
        loss = svi.step(X)
        losses.append(loss)
        t.set_postfix(loss=loss)
    return (svi, losses)

In [24]:
svi, losses = train(X_class_0)

ValueError: at site "probs", invalid log_prob shape
  Expected [], actual [59, 13]
  Try one of the following fixes:
  - enclose the batched tensor in a with plate(...): context
  - .to_event(...) the distribution being sampled
  - .permute() data dimensions

In [7]:
pyro_mu = pyro.param("mu")
pyro_sigma = pyro.param("sigma")
data_mu = X_class_0.mean(0)
data_sigma = X_class_0.std(0)

In [8]:
print(data_mu)
print(pyro_mu)
print(pyro_mu - data_mu)
print()
print(data_sigma)
print(pyro_sigma)
print(pyro_sigma - data_sigma)

tensor([1.3745e+01, 2.0107e+00, 2.4556e+00, 1.7037e+01, 1.0634e+02, 2.8402e+00,
        2.9824e+00, 2.9000e-01, 1.8993e+00, 5.5283e+00, 1.0620e+00, 3.1578e+00,
        1.1157e+03], dtype=torch.float64)
tensor([5.9885, 2.0107, 2.4556, 4.8188, 3.5172, 2.8402, 2.9824, 0.2900, 1.8993,
        5.5283, 1.0620, 3.1578, 3.4797], dtype=torch.float64,
       requires_grad=True)
tensor([-7.7562e+00, -4.4409e-16, -4.4409e-16, -1.2218e+01, -1.0282e+02,
         8.8818e-16,  0.0000e+00, -1.1842e-07, -2.2204e-16,  3.5527e-15,
         0.0000e+00,  0.0000e+00, -1.1122e+03], dtype=torch.float64,
       grad_fn=<SubBackward0>)

tensor([4.6213e-01, 6.8855e-01, 2.2717e-01, 2.5463e+00, 1.0499e+01, 3.3896e-01,
        3.9749e-01, 7.0049e-02, 4.1211e-01, 1.2386e+00, 1.1648e-01, 3.5708e-01,
        2.2152e+02], dtype=torch.float64)
tensor([ 8.5325,  0.6885,  0.2272, 13.0133, 30.8680,  0.3390,  0.3975,  0.0700,
         0.4121,  1.2386,  0.1165,  0.3571, 32.3126], dtype=torch.float64,
       grad_fn=<AddBackwa